In [2]:
import pandas as pd
import numpy as np
import yaml
from tqdm import tqdm
from pathlib import Path
from glob import glob
from functools import partial
from multiprocessing import Pool

import shapefile
import shapely
from shapely.geometry import Point

pd.set_option('max_columns', None)

In [2]:
df = pd.read_csv('../Data/Precipitaciones/cru_80-20.csv')
print(df.shape)
print('Unique locs', df.drop_duplicates(['lon', 'lat']).shape)
df.head()

(67420, 494)
Unique locs (67420, 494)


,lon,lat,80-01,80-02,80-03,80-04,80-05,80-06,80-07,80-08,80-09,80-10,80-11,80-12,81-01,81-02,81-03,81-04,81-05,81-06,81-07,81-08,81-09,81-10,81-11,81-12,82-01,82-02,82-03,82-04,82-05,82-06,82-07,82-08,82-09,82-10,82-11,82-12,83-01,83-02,83-03,83-04,83-05,83-06,83-07,83-08,83-09,83-10,83-11,83-12,84-01,84-02,84-03,84-04,84-05,84-06,84-07,84-08,84-09,84-10,84-11,84-12,85-01,85-02,85-03,85-04,85-05,85-06,85-07,85-08,85-09,85-10,85-11,85-12,86-01,86-02,86-03,86-04,86-05,86-06,86-07,86-08,86-09,86-10,86-11,86-12,87-01,87-02,87-03,87-04,87-05,87-06,87-07,87-08,87-09,87-10,87-11,87-12,88-01,88-02,88-03,88-04,88-05,88-06,88-07,88-08,88-09,88-10,88-11,88-12,89-01,89-02,89-03,89-04,89-05,89-06,89-07,89-08,89-09,89-10,89-11,89-12,90-01,90-02,90-03,90-04,90-05,90-06,90-07,90-08,90-09,90-10,90-11,90-12,91-01,91-02,91-03,91-04,91-05,91-06,91-07,91-08,91-09,91-10,91-11,91-12,92-01,92-02,92-03,92-04,92-05,92-06,92-07,92-08,92-09,92-10,92-11,92-12,93-01,93-02,93-03,93-04,93-05,93-06,93-07,93-08,93-09,93-10,93-11,93-12,94-01,94-02,94-03,94-04,94-05,94-06,94-07,94-08,94-09,94-10,94-11,94-12,95-01,95-02,95-03,95-04,95-05,95-06,95-07,95-08,95-09,95-10,95-11,95-12,96-01,96-02,96-03,96-04,96-05,96-06,96-07,96-08,96-09,96-10,96-11,96-12,97-01,97-02,97-03,97-04,97-05,97-06,97-07,97-08,97-09,97-10,97-11,97-12,98-01,98-02,98-03,98-04,98-05,98-06,98-07,98-08,98-09,98-10,98-11,98-12,99-01,99-02,99-03,99-04,99-05,99-06,99-07,99-08,99-09,99-10,99-11,99-12,00-01,00-02,00-03,00-04,00-05,00-06,00-07,00-08,00-09,00-10,00-11,00-12,01-01,01-02,01-03,01-04,01-05,01-06,01-07,01-08,01-09,01-10,01-11,01-12,02-01,02-02,02-03,02-04,02-05,02-06,02-07,02-08,02-09,02-10,02-11,02-12,03-01,03-02,03-03,03-04,03-05,03-06,03-07,03-08,03-09,03-10,03-11,03-12,04-01,04-02,04-03,04-04,04-05,04-06,04-07,04-08,04-09,04-10,04-11,04-12,05-01,05-02,05-03,05-04,05-05,05-06,05-07,05-08,05-09,05-10,05-11,05-12,06-01,06-02,06-03,06-04,06-05,06-06,06-07,06-08,06-09,06-10,06-11,06-12,07-01,07-02,07-03,07-04,07-05,07-06,07-07,07-08,07-09,07-10,07-11,07-12,08-01,08-02,08-03,08-04,08-05,08-06,08-07,08-08,08-09,08-10,08-11,08-12,09-01,09-02,09-03,09-04,09-05,09-06,09-07,09-08,09-09,09-10,09-11,09-12,10-01,10-02,10-03,10-04,10-05,10-06,10-07,10-08,10-09,10-10,10-11,10-12,11-01,11-02,11-03,11-04,11-05,11-06,11-07,11-08,11-09,11-10,11-11,11-12,12-01,12-02,12-03,12-04,12-05,12-06,12-07,12-08,12-09,12-10,12-11,12-12,13-01,13-02,13-03,13-04,13-05,13-06,13-07,13-08,13-09,13-10,13-11,13-12,14-01,14-02,14-03,14-04,14-05,14-06,14-07,14-08,14-09,14-10,14-11,14-12,15-01,15-02,15-03,15-04,15-05,15-06,15-07,15-08,15-09,15-10,15-11,15-12,16-01,16-02,16-03,16-04,16-05,16-06,16-07,16-08,16-09,16-10,16-11,16-12,17-01,17-02,17-03,17-04,17-05,17-06,17-07,17-08,17-09,17-10,17-11,17-12,18-01,18-02,18-03,18-04,18-05,18-06,18-07,18-08,18-09,18-10,18-11,18-12,19-01,19-02,19-03,19-04,19-05,19-06,19-07,19-08,19-09,19-10,19-11,19-12,20-01,20-02,20-03,20-04,20-05,20-06,20-07,20-08,20-09,20-10,20-11,20-12
0,-68.25,-55.75,59.000000,141.900009,135.100006,51.200001,59.200001,75.000000,107.200005,69.500000,103.000000,73.700005,66.000000,60.000000,61.400002,67.900002,241.699997,98.400002,125.500000,97.300003,136.000000,141.100006,87.000000,72.300003,41.500000,76.700005,92.700005,99.000000,57.100002,70.900002,64.900002,58.799999,84.200005,46.400002,72.099998,134.800003,83.000000,43.200001,81.400002,90.200005,80.900002,78.500000,67.000000,78.900002,32.000000,64.000000,36.400002,59.799999,45.200001,28.200001,88.900002,52.700001,59.400002,104.300003,160.900009,59.400002,67.599998,80.400002,39.799999,85.000000,45.299999,40.100002,55.200001,73.700005,96.700005,50.000000,50.000000,61.799999,34.200001,37.799999,82.000000,45.600002,78.200005,73.800003,41.100002,93.000000,67.500000,72.800003,122.099998,103.300003,104.599998,29.000000,45.500000,42.100002,116.599998,90.500000,92.099998,111.800003,64.300003,104.200005,42.600002,89.300003,112.700005,31.400000,60.200001,69.599998,73.099998,54.400002,94.599998,77.800003,94.200005,90.900002,49.50

In [3]:
shps = glob('../Shapefiles/preprocessed/*.shp')
shapes_by_country = {}
for i, shp in tqdm(enumerate(shps), total=len(shps)):
    codes = Path(shp).name.replace('.shp', '').split('_')
    shp = shapefile.Reader(shp)
    shapes = [shapely.geometry.shape(s) for s in shp.shapes()]
    records = shp.records()
    
    for code in codes:
        shapes_by_country[code] = []
        for shape, record in zip(shapes, records):
            shapes_by_country[code].append((shape, *record))
    
assert 'GT' in shapes_by_country and 'PM' in shapes_by_country

100%|██████████| 276/276 [02:15<00:00,  2.03it/s]


In [6]:
lon = 'lon'
lat = 'lat'
country = 'country'
locs = df  # [[lon, lat]].drop_duplicates()
locs['adm1'] = np.nan
locs['adm2'] = np.nan
locs['nearest_loc'] = False

In [9]:
def country_shapes(country=None):
    if country is not None and country in shapes_by_country:
        print('Found country', country)
        yield from shapes_by_country[country]
    else:
        for shapes in shapes_by_country.values():
            yield from shapes
            
            
def match_single_geo(row, lon_col, lat_col, country_col):
    if isinstance(row, tuple):
        row = row[1]
        
    if isinstance(row['adm1'], str):
        return {'nearest_loc': row['nearest_loc'], 
                'adm1': row['adm1'],
                'adm2': row['adm2'], 
                'adm0': row['adm0'],
                'idx': row.name}
    
    results = {'nearest_loc': False, 'idx': row.name}

    point = Point((float(row[lon_col]), float(row[lat_col])))
    nearest_record = None
    nearest_distance = np.inf
    for shape, country, adm1, adm2 in country_shapes(row[country_col]):
        if point.within(shape):
            results['adm0'] = country
            results['adm2'] = adm2
            results['adm1'] = adm1
            break
    else:
        for shape, country, adm1, adm2 in country_shapes(row[country_col]):
            distance = point.distance(shape)
            if distance < nearest_distance:
                nearest_distance = distance
                nearest_record = record

        if nearest_record:
            results['nearest_loc'] = True
            results['adm0'] = country
            results['adm2'] = adm2
            results['adm1'] = adm1
    
    return results    
            
            
def match_geo(locs, lon_col, lat_col, country_col, pool_size=30):
    f = partial(match_single_geo, lon_col=lon_col, lat_col=lat_col, country_col=country_col)
    
    if pool_size > 1:
        with Pool(pool_size) as p:
            results = results = [x for x in tqdm(p.imap_unordered(f, locs.iterrows()), total=locs.shape[0])]
    else:
        results = [f(x) for x in locs.iterrows()]
    
    for result in results:
        idx = result['idx']
        locs.loc[idx, 'adm2'] = result.get('adm2')
        locs.loc[idx, 'adm1'] = result.get('adm1')
        locs.loc[idx, 'adm0'] = result.get('adm0')
        locs.loc[idx, 'nearest_loc'] = result['nearest_loc']
        
    return locs


locs['country'] = 'no country'

locs = match_geo(locs, lon, lat, country, pool_size=30)
print('Total nearest:', locs['nearest_loc'].sum())

Total nearest: 5899


KeyError: 'CTR_MN_ISO'

In [11]:
print(df.shape)
df.drop(columns='country', inplace=True)
df.to_csv('../Output/Precipitaciones/cru.csv', index=False)

(67420, 499)


# Aggregate

In [3]:
def clean_time(df, base_year=1900):
    time = df['time'].str.split('-', expand=True)
    time.columns = ['year', 'month']
    time['year'] = pd.to_numeric(time['year'])
    
    time['year'] = np.where(time['year'] > 21, base_year + time['year'], base_year + 100 + time['year'])
    time['month'] = pd.to_numeric(time['month'])
    df['year'] = time['year']
    df['month'] = time['month']

time_groups = {'yearly': ['year'], 'monthly': ['year', 'month']}
loc_groups = {'country': ['adm0'], 
              'edo': ['adm0', 'adm1'], 
              'mun': ['adm0', 'adm1', 'adm2']}

groups = {}
for loc_name, loc_group in loc_groups.items():
    for time_name, time_group in time_groups.items():
        groups[f'{loc_name}_{time_name}'] = loc_group + time_group

output_folder = Path('../Output/Precipitaciones/CRU')
done = [Path(x).name for x in glob(str(output_folder / 'CRU_*.csv'))]

files = ['../Output/Precipitaciones/cru.csv']
for i, file in enumerate(files):
    print('Reading', file)
    df = pd.read_csv(file)
    df = df.melt(id_vars=['lon', 'lat', 'adm0', 'adm1', 'adm2', 'nearest_loc'],
                 var_name='time')
    
    print('Getting time')
    base_year = 1800 if '1891' in file else 1900
    clean_time(df, base_year)
    
    for group_name, group in groups.items():
        fname = f'CRU_{group_name}_b{i}.csv'
        if fname in done:
            continue
            
        print(group_name)
        n_groups = len(df.drop_duplicates(group))
        collapsed = []
        
        pivot = pd.pivot_table(df, 
                               values=['value', 'nearest_loc'],
                               index=group,
                               aggfunc={'value': [np.mean, np.median, np.std, 'count'], 'nearest_loc': np.mean})
        pivot.to_csv(output_folder / fname)
        
        pivot = pd.pivot_table(df[~df['nearest_loc']], 
                               values=['value'],
                               index=group,
                               aggfunc={'value': [np.mean, np.median, np.std, 'count']})
        pivot['nearest_loc_mean'] = 0
        pivot.to_csv(output_folder / f'CRU_nnl_{group_name}_b{i}.csv')
        
    del df

Reading ../Output/Precipitaciones/cru.csv
Getting time
country_yearly
country_monthly
edo_yearly
edo_monthly
mun_yearly
mun_monthly
